# Connect to Chatgpt API to collect entities and relations

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv()  # loads variables from .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [2]:
# Quick test: Ask model for 5 words
resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say 5 random fruits"}],
    temperature=0.1
)

print(resp.choices[0].message.content)

Sure! Here are five random fruits:

1. Mango
2. Kiwi
3. Dragon fruit
4. Blueberry
5. Papaya


In [3]:
# Load your Excel file
df = pd.read_excel("/Users/alexa/Projects/cdmx_kg/Mexico_City/LEY_AMBIENTAL_DE_LA_CIUDAD_DE_MEXICO.xlsx")
df = df[['row_id','doc_hash','document_name', 'document_section_title', 'text']]
print(df.head())

              row_id      doc_hash                         document_name  \
0  12406E12_ARTCULO1  1.240600e+16  LEY AMBIENTAL DE LA CIUDAD DE MÉXICO   
1  12406E12_ARTCULO2  1.240600e+16  LEY AMBIENTAL DE LA CIUDAD DE MÉXICO   
2  12406E12_ARTCULO3  1.240600e+16  LEY AMBIENTAL DE LA CIUDAD DE MÉXICO   
3  12406E12_ARTCULO4  1.240600e+16  LEY AMBIENTAL DE LA CIUDAD DE MÉXICO   
4  12406E12_ARTCULO5  1.240600e+16  LEY AMBIENTAL DE LA CIUDAD DE MÉXICO   

  document_section_title                                               text  
0             Artículo 1  º.- La presente Ley es reglamentaria de las di...  
1             Artículo 2  º.- Se consideran de utilidad pública:\n\nI. E...  
2             Artículo 3  º.- En todo lo no previsto en la presente Ley,...  
3             Artículo 4  º.- Para los efectos de esta Ley, se utilizará...  
4             Artículo 5  º.- Son autoridades en materia ambiental en la...  


In [10]:
# 1. Firstly ask to identify entities. 
# Output is a csv with a art_name | mention | 

# Prompts
def prompt_entities(batch):
    text = "\n\n".join(
        f"ROW_ID: {row.row_id}\n"
        f"DOCUMENT: {row.document_name}\n"
        f"SECTION: {row.document_section_title}\n"
        f"TEXT: {row.text}"
        for row in batch.itertuples()
    )
    return f"""You are a specialized legal entity extraction system for Mexico City government documents. Your task is to be EXHAUSTIVE and find ALL government entities mentioned.

CRITICAL: READ EVERY WORD CAREFULLY. Do not miss any government entity, no matter how briefly mentioned.

TASK: Extract ALL government entities, institutions, and legal bodies from these legal texts.

ENTITY TYPES TO IDENTIFY (scan for ALL of these):
- Secretarías (ministries): Secretaría de..., SEDEMA, SEDUVI, SIBISO, STyFE, etc.
- Alcaldías (municipal governments): Alcaldía..., any of the 16 alcaldías
- Institutos and agencies: Instituto de..., Agencia de..., ADIP, etc.
- Tribunales and courts: Tribunal..., Juzgado..., Corte..., etc.
- Consejos and commissions: Consejo de..., Comisión de..., etc.
- Universities and schools: Universidad de..., UACM, etc.
- Procuradurías: Procuraduría..., PAOT, etc.
- Federal entities: SEMARNAT, CONAGUA, PROFEPA, etc.
- Any organization with official government role

COMPREHENSIVE EXTRACTION RULES:
1. Extract EXACTLY as written in the text (preserve case, accents, articles like "la", "el")
2. Include full official names AND abbreviations when mentioned
3. Look for entities in ALL parts of the text, including parentheses, footnotes, lists
4. Capture entities mentioned in different forms (e.g., "la Secretaría", "dicha Secretaría")
5. Include entities that appear in compound phrases
6. Do NOT skip vague references - if it's a government body, include it
7. Be especially careful with lists and enumerated items

SEARCH STRATEGY:
- Read the text word by word
- Look for capital letters that might indicate proper nouns
- Check for organizational keywords: Secretaría, Instituto, Consejo, Comisión, Alcaldía, etc.
- Examine abbreviations in parentheses
- Review any lists or bullet points carefully

OUTPUT FORMAT - CSV rows only, no headers, no markdown blocks:
row_id,mention

EXAMPLES:
12406E12_ARTCULO5,Secretaría del Medio Ambiente
12406E12_ARTCULO5,SEDEMA
12406E12_ARTCULO5,Jefatura de Gobierno de la Ciudad de México
12406E12_ARTCULO5,la Secretaría
12406E12_ARTCULO5,Alcaldía Benito Juárez

INPUT:
{text}
"""


In [11]:
# 2. Secondly ask to identify article mentions. 
#Each article mention must include the number article and the law.
# Output is a csv with art_name | art_mention 

def prompt_article_mentions(batch):
    text = "\n\n".join(
        f"ROW_ID: {row.row_id}\n"
        f"DOCUMENT: {row.document_name}\n"
        f"SECTION: {row.document_section_title}\n"
        f"TEXT: {row.text}"
        for row in batch.itertuples()
    )
    return f"""
Extract ALL laws and articles of laws mentions from Mexican law documents. Analyze ONLY the TEXT field.

CRITICAL RULES:
• ONLY extract actual references to LAWS, ARTICLES, or LEGAL DOCUMENTS. 
• DO NOT extract mere mentions of government entities, authorities, or institutions.
• For "artículo anterior": use current article number -1, same law
• For "esta Ley"/"presente Ley": use law name from DOCUMENT field
• For article WITHOUT law mentioned: assume it refers to current law from DOCUMENT field
• For "reglamento": format as "REGLAMENTO DE + [DOCUMENT]"


EXTRACT:
• Articles: "artículo 123", "art. 45", "artículos 1 al 15"
• Laws: "Ley de...", "Código Civil", "Constitución Política", "Reglamento de..."
• Self-references: "esta Ley", "la presente Ley"
• Relative refs: "artículo anterior", "artículo siguiente"

RULES:
1. One row per mention (split ranges: "artículos 13 y 16" = 2 rows)
2. Include complete phrase in mention_extraction and 10 words before and after
3. For ranges/multiple: create separate rows
4. Check parentheses, lists, subordinate clauses
5. Law names can be long with commas (e.g., "LEY DEL DERECHO AL ACCESO, DISPOSICIÓN Y SANEAMIENTO DEL AGUA DE LA CIUDAD DE MÉXICO")


OUTPUT (CSV only, no headers):
row_id,art_num,law_name,mention_extraction

COLUMN DEFINITIONS:
• row_id: The ROW_ID of the current article being analyzed (from input)
• art_num: Article number being referenced (leave EMPTY if no article number)
• law_name: Name of the law/document containing the referenced article
• mention_extraction: Complete text phrase of the legal mention and 10 words before and after as it appears in the text

EXAMPLES:
Text: "artículo 13 de la Constitución Política" → 12406E12_ARTCULO1,13,CONSTITUCIÓN POLÍTICA,artículo 13 de la Constitución Política
Text: "el Código Civil" → 12406E12_ARTCULO3,,CÓDIGO CIVIL,el Código Civil  
Text: "artículo anterior" (in ARTÍCULO 5) → 12406E12_ARTCULO5,4,LEY AMBIENTAL DE LA CIUDAD DE MÉXICO,artículo anterior
Text: "artículo 10" (no law mentioned, current law is LEY AMBIENTAL) → 12406E12_ARTCULO5,10,LEY AMBIENTAL DE LA CIUDAD DE MÉXICO,artículo 10
Text: "su reglamento" (current law is LEY AMBIENTAL) → 12406E12_ARTCULO5,,REGLAMENTO DE LA LEY AMBIENTAL DE LA CIUDAD DE MÉXICO,su reglamento

INPUT:
{text}
"""

In [ ]:
# Helper function to clean and validate GPT output
def clean_csv_output(raw_output, expected_columns):
    """Clean and validate CSV output from GPT, filtering out artifacts and malformed entries"""
    clean_lines = []
    for line in raw_output.strip().splitlines():
        line = line.strip()
        
        # Skip empty lines
        if not line:
            continue
            
        # Skip markdown code blocks
        if line.startswith('```'):
            continue
            
        # Skip headers (case insensitive)
        if line.lower().startswith('art_name') or line.lower().startswith('document'):
            continue
            
        # Skip lines that look like instructions or comments
        if line.startswith('#') or line.startswith('//') or line.startswith('OUTPUT') or line.startswith('EXAMPLES'):
            continue
            
        # Must contain commas for CSV format
        if ',' not in line:
            continue
            
        # Check if line has the expected number of columns (allow some flexibility)
        parts = line.split(',')
        if len(parts) < expected_columns:
            print(f"    Skipping malformed line (too few columns): {line[:50]}...")
            continue
            
        # Clean each part
        cleaned_parts = []
        for part in parts[:expected_columns]:  # Only take expected number of columns
            cleaned_part = part.strip().strip('"').strip("'")  # Remove quotes and extra spaces
            cleaned_parts.append(cleaned_part)
            
        # Skip if any essential fields are empty (first two columns should not be empty)
        if not cleaned_parts[0]:
            print(f"    Skipping line with empty essential fields: {line[:50]}...")
            continue
            
        # Validate row_id structure (should match pattern like: 12406E12_ARTCULO5)
        import re
        row_id_pattern = r'^[A-F0-9]{8}_[A-ZÁÉÍÓÚÑÜ]+\d*$'
        if not re.match(row_id_pattern, cleaned_parts[0]):
            print(f"    Skipping line with invalid row_id structure: {line[:50]}...")
            continue
            
        clean_lines.append(cleaned_parts)
    
    return clean_lines

# Collectors
gov_entities = []
article_mentions = []

# Batch size - reduced for better quality and completeness
BATCH_SIZE = 8  # Even smaller batches for better attention

total_batches = (len(df) + BATCH_SIZE - 1) // BATCH_SIZE
print(f"Processing {len(df)} articles in {total_batches} batches...")

for i, start in enumerate(range(0, len(df), BATCH_SIZE)):
    batch = df.iloc[start:start+BATCH_SIZE]
    batch_num = i + 1
    
    print(f"\nProcessing batch {batch_num}/{total_batches} (articles {start+1}-{min(start+BATCH_SIZE, len(df))})")

    # # Step 1 – Entities (with dual-pass for completeness)
    # print("  → Extracting government entities...")
    # try:
    #     # First pass - standard extraction
    #     resp1 = client.chat.completions.create(
    #         model="gpt-4o-mini",
    #         messages=[{"role": "user", "content": prompt_entities(batch)}],
    #         temperature=0.1
    #     )
    #     output1 = resp1.choices[0].message.content.strip()
        
    #     # Second pass - with different temperature for variation
    #     resp1_alt = client.chat.completions.create(
    #         model="gpt-4o-mini",
    #         messages=[{"role": "user", "content": prompt_entities(batch)}],
    #         temperature=0.3  # Higher temperature for different perspective
    #     )
    #     output1_alt = resp1_alt.choices[0].message.content.strip()
        
    #     # Combine both outputs
    #     combined_entities = []
    #     if output1:
    #         cleaned_entities_1 = clean_csv_output(output1, 2)
    #         combined_entities.extend(cleaned_entities_1)
    #     if output1_alt:
    #         cleaned_entities_2 = clean_csv_output(output1_alt, 2)
    #         combined_entities.extend(cleaned_entities_2)
        
    #     # Remove duplicates while preserving order
    #     seen = set()
    #     unique_entities = []
    #     for entity in combined_entities:
    #         entity_key = tuple(entity)
    #         if entity_key not in seen:
    #             seen.add(entity_key)
    #             unique_entities.append(entity)
        
    #     gov_entities.extend(unique_entities)
    #     print(f"    Found {len(unique_entities)} unique entities (from {len(combined_entities)} total)")
            
    # except Exception as e:
    #     print(f"    Error extracting entities: {e}")

    # Step 2 – Article mentions (with dual-pass for completeness)
    print("  → Extracting article mentions...")
    try:
        # First pass - standard extraction
        resp2 = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt_article_mentions(batch)}],
            temperature=0.1
        )
        output2 = resp2.choices[0].message.content.strip()
        
        # Second pass - with different temperature for variation
        resp2_alt = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt_article_mentions(batch)}],
            temperature=0.3  # Higher temperature for different perspective
        )
        output2_alt = resp2_alt.choices[0].message.content.strip()
        
        # Combine both outputs
        combined_mentions = []
        if output2:
            cleaned_mentions_1 = clean_csv_output(output2, 4)
            combined_mentions.extend(cleaned_mentions_1)
        if output2_alt:
            cleaned_mentions_2 = clean_csv_output(output2_alt, 4)
            combined_mentions.extend(cleaned_mentions_2)
        
        # Remove duplicates while preserving order
        seen = set()
        unique_mentions = []
        for mention in combined_mentions:
            mention_key = tuple(mention)
            if mention_key not in seen:
                seen.add(mention_key)
                unique_mentions.append(mention)
        
        article_mentions.extend(unique_mentions)
        print(f"    Found {len(unique_mentions)} unique mentions (from {len(combined_mentions)} total)")
            
    except Exception as e:
        print(f"    Error extracting mentions: {e}")
    
    # Small delay between batches to avoid rate limits (now with 4 API calls per batch)
    import time
    time.sleep(0.5)

print(f"\n=== EXTRACTION COMPLETE ===")
print(f"Total government entities found: {len(gov_entities)}")
print(f"Total article mentions found: {len(article_mentions)}")

# Quality summary
print(f"\n=== QUALITY METRICS ===")
if gov_entities:
    unique_entity_mentions = len(set(tuple(e) for e in gov_entities))
    print(f"Unique government entities: {unique_entity_mentions}")
if article_mentions:
    unique_article_mentions = len(set(tuple(m) for m in article_mentions))
    print(f"Unique article mentions: {unique_article_mentions}")




Processing 332 articles in 42 batches...

Processing batch 1/42 (articles 1-8)
  → Extracting article mentions...
    Found 20 unique mentions (from 23 total)

Processing batch 2/42 (articles 9-16)
  → Extracting article mentions...
    Found 11 unique mentions (from 16 total)

Processing batch 3/42 (articles 17-24)
  → Extracting article mentions...
    Found 8 unique mentions (from 16 total)

Processing batch 4/42 (articles 25-32)
  → Extracting article mentions...
    Found 35 unique mentions (from 43 total)

Processing batch 5/42 (articles 33-40)
  → Extracting article mentions...
    Found 17 unique mentions (from 20 total)

Processing batch 6/42 (articles 41-48)
  → Extracting article mentions...
    Found 15 unique mentions (from 17 total)

Processing batch 7/42 (articles 49-56)
  → Extracting article mentions...
    Found 13 unique mentions (from 19 total)

Processing batch 8/42 (articles 57-64)
  → Extracting article mentions...
    Found 13 unique mentions (from 21 total)

Pr

In [7]:
# Save results with proper validation
# if gov_entities:
#     entities_df = pd.DataFrame(gov_entities, columns=["row_id", "mention"])
#     entities_df.to_csv("gov_entities.csv", index=False, encoding='utf-8')
#     print(f"✅ Saved government entities to gov_entities.csv")
# else:
#     print("⚠️  No government entities to save")

if article_mentions:
    mentions_df = pd.DataFrame(article_mentions, columns=["row_id", "art_num", "law_name", "mention_extraction"])
    mentions_df.to_csv("/Users/alexa/Projects/cdmx_kg/data/article_mentions_gpt.csv", index=False, encoding='utf-8')
    print(f"✅ Saved article mentions to article_mentions.csv")
else:
    print("⚠️  No article mentions to save")

✅ Saved article mentions to article_mentions.csv


In [ ]:
# Run the entity normalization and ID system
mentions_df, entities_df, normalization_map = create_entity_id_system()


# This code section is to use the output of of the identified mentions and classfy them 

In [ ]:
# For government entities: 
# Read the dictionary of the document.
# Then, go to the list of entities and check if it is needed to change the mention name based on the dictionary.
# Replace the name if needed.
# Not all the mentions will be needed to be changed, only ones specifiqued for each document. 

# Then, look at the list of mentions and create a new list with the id_hash for each differnte entity.
# Add the hash_id to the list of mentions based on the entity name. 
# The output is the list of mentions with the hash_id for each entity and the list of unique entities with the hash_id.



